# Bidirection LSTM - IMDB sentiment classification

see **https://github.com/fchollet/keras/blob/master/examples/imdb_bidirectional_lstm.py**

In [1]:
WEIGHTS_FILEPATH = 'imdb_bidirectional_lstm.hdf5'
MODEL_ARCH_FILEPATH = 'imdb_bidirectional_lstm.json'

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional
from keras.datasets import imdb
from keras.callbacks import EarlyStopping, ModelCheckpoint

import json

Using TensorFlow backend.


In [3]:
max_features = 20
maxlen = 200  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

X_train = X_train[:1000]
y_train = y_train[:1000]

X_test = X_test[:1000]
y_test = y_test[:1000]
print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Loading data...
(25000, 'train sequences')
(25000, 'test sequences')
Pad sequences (samples x time)
('X_train shape:', (1000, 200))
('X_test shape:', (1000, 200))


In [4]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [5]:
# Model saving callback
#checkpointer = ModelCheckpoint(filepath=WEIGHTS_FILEPATH, monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
#early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=2)

# train
batch_size = 32
nb_epoch = 10
model.fit(X_train, y_train, 
          validation_data=[X_test, y_test],
          batch_size=batch_size, nb_epoch=nb_epoch, verbose=1) #,          callbacks=[checkpointer, early_stopping])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 39s - loss: 0.6949 - acc: 0.5030 - val_loss: 0.6926 - val_acc: 0.5140
Epoch 2/10
1000/1000 [==============================] - 39s - loss: 0.6941 - acc: 0.5090 - val_loss: 0.6931 - val_acc: 0.5080
Epoch 3/10
1000/1000 [==============================] - 38s - loss: 0.6938 - acc: 0.5020 - val_loss: 0.6927 - val_acc: 0.5280
Epoch 4/10
1000/1000 [==============================] - 38s - loss: 0.6930 - acc: 0.5060 - val_loss: 0.6925 - val_acc: 0.5250
Epoch 5/10
1000/1000 [==============================] - 37s - loss: 0.6903 - acc: 0.5320 - val_loss: 0.6929 - val_acc: 0.5210
Epoch 6/10
1000/1000 [==============================] - 37s - loss: 0.6874 - acc: 0.5700 - val_loss: 0.6936 - val_acc: 0.4940
Epoch 7/10
1000/1000 [==============================] - 37s - loss: 0.6839 - acc: 0.5600 - val_loss: 0.6934 - val_acc: 0.5130
Epoch 8/10
1000/1000 [==============================] - 37s - loss: 0.

In [6]:
with open(MODEL_ARCH_FILEPATH, 'w') as f:
    f.write(model.to_json())

**sample data**

In [7]:
word_index = imdb.get_word_index()

IOError: [Errno 13] Permission denied: '/home/ubuntu/.keras/datasets/imdb_word_index.pkl'

In [ ]:
word_dict = {idx: word for word, idx in word_index.items()}

In [ ]:
sample = []
for idx in X_train[0]:
    if idx >= 3:
        sample.append(word_dict[idx-3])
    elif idx == 2:
        sample.append('-')
' '.join(sample)

In [ ]:
with open('imdb_dataset_word_index_top20000.json', 'w') as f:
    f.write(json.dumps({word: idx for word, idx in word_index.items() if idx < max_features}))

In [ ]:
with open('imdb_dataset_word_dict_top20000.json', 'w') as f:
    f.write(json.dumps({idx: word for word, idx in word_index.items() if idx < max_features}))

In [ ]:
sample_test_data = []
for i in np.random.choice(range(X_test.shape[0]), size=1000, replace=False):
    sample_test_data.append({'values': X_test[i].tolist(), 'label': y_test[i].tolist()})
    
with open('imdb_dataset_test.json', 'w') as f:
    f.write(json.dumps(sample_test_data))